In [26]:
data_filepath = 'the average human being - old data.csv'

import markdown
def fact_page_html(url='', fact='', source='', note='', url_list=[], start_page=False, **kwargs):
    # check that inputs are correct types
    if not isinstance(url, str):
        raise TypeError('url needs to be a string')
    if not isinstance(fact, str):
        raise TypeError('fact needs to be a string')
    if not isinstance(source, str):
        raise TypeError('source needs to be a string')
    if not isinstance(note, str):
        raise TypeError('note needs to be a string')
    if not isinstance(url_list, list):
        raise TypeError('url_list needs to be a list of strings')
    if not isinstance(start_page, bool):
        raise TypeError('start_page needs to be True or False')

    # check that required arguments are present
    if not url:
        raise ValueError('url is required')
    if not url_list:
        raise ValueError('url_list is required')

    # compute intermediate html chunks
    if source:
        source_html = f'<div><h2>Source:</h2> {markdown.markdown(source)}</div>'
    else:
        source_html = ''

    if note:
        note_html = f'<div><h2>Note:</h2> {markdown.markdown(note)}</div>'
    else:
        note_html = ''

    title = f'The average human being {fact}'

    if start_page:
        start_page_script = f"""\
            <script>
                var urls = {url_list};
                location.href = urls[Math.floor(Math.random() * urls.length)];
            </script>
            """
        import textwrap
        start_page_script = textwrap.dedent(start_page_script)
    else:
        start_page_script = ''


    # load template
    with open('fact_page_template.html', 'r') as fact_page_template_file:
        fact_page_template_string = fact_page_template_file.read()

    # return completed template
    return fact_page_template_string.format(
        url=url, 
        fact=fact, 
        title=title, 
        source_html=source_html, 
        note_html=note_html, 
        url_list=url_list,
        start_page_script=start_page_script
        )


import csv
with open(data_filepath, 'r') as data_file:
    # read csv, load all rows
    reader = csv.DictReader(data_file, delimiter=',')
    fact_dicts = [fact_dict for fact_dict in reader]

    # create url list and insert into fact dicts
    url_list = [fact_dict['url'] for fact_dict in fact_dicts]
    for fact_dict in fact_dicts:
        url_list_without_self = [url for url in url_list if url != fact_dict['url']]
        fact_dict['url_list'] = url_list_without_self

    # create start page
    with open('index.html', 'w') as start_page:
        start_page.write(fact_page_html(url='index.html', url_list=url_list, start_page=True))

    # convert urls - PUT AFTER PAGE CREATION IF YOU WANT LINKS WITHOUT HTML SUFFIX
    for fact_dict in fact_dicts:
        fact_dict['url'] = fact_dict['url'] + '.html'

    # create pages
    for fact_dict in fact_dicts:
        with open(fact_dict['url'], 'w') as fact_page:
            fact_page.write(fact_page_html(**fact_dict))
print(f'{1+len(fact_dicts)} pages created:')
print(['index.html'] + url_list)

6 pages created:
['index.html', 'grapes.html', 'wine_grapes.html', 'table_grapes.html', 'dried_grapes.html', 'vineyards.html']
